In [1]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.
   ━━━━━━━━

In [2]:
!pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/bi_influence.git

  Cloning https://github.com/Shannu3766/bi_influence.git to /tmp/pip-req-build-gdk7qxxs
  Running command git clone --filter=blob:none --quiet https://github.com/Shannu3766/bi_influence.git /tmp/pip-req-build-gdk7qxxs
  Resolved https://github.com/Shannu3766/bi_influence.git to commit 0bfad85a0a373b96ec3af51160622924d109e3be
  Preparing metadata (setup.py) ... done
  Created wheel for adaptive_lora: filename=adaptive_lora-2.1.0-py3-none-any.whl size=7940 sha256=7e936cb499f618bafab15c1d7ad10d98332d3aaeb61339f597c6a3bb587e4bcd
  Stored in directory: /tmp/pip-ephem-wheel-cache-fasrbb88/wheels/4b/91/1d/e153ff3aa3759d6ccdcc4706d8559f1cbfaa4aa22b992958cd
Successfully built adaptive_lora


In [3]:
import warnings
import os
import logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
# logging.set_verbosity_error()

In [4]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from torch.utils.data import DataLoader
from adaptive_lora.callbacks import AdaptiveLoRACallback
import numpy as np
print("torch:", torch.__version__)
print("cuda devices:", torch.cuda.device_count())

2025-11-23 17:01:24.661628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763917284.853104      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763917284.906646      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

torch: 2.6.0+cu124
cuda devices: 2


In [5]:
import os
import warnings
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    logging
)
from peft import get_peft_model, LoraConfig
import evaluate

In [6]:
# Silence all unnecessary outputs
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
logging.set_verbosity_error()


In [7]:
dataset = load_dataset("glue", "sst2")

# 2️⃣ Model checkpoint and labels
model_checkpoint = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

# 3️⃣ Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    device_map="auto"
)

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [8]:
# 4️⃣ Load tokenizer and fix pad token
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

In [9]:
def tokenize_function(examples):
    return tokenizer(
        examples["sentence"],
        truncation=True,
        padding=False,  # ✅ Don't pad here — DataCollator will pad later
        max_length=512
    )

In [10]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["sentence", "idx"])

# 6️⃣ Data collator handles dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [11]:
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return {"accuracy": accuracy.compute(predictions=preds, references=labels)}

# 8️⃣ LoRA config
peft_config = LoraConfig(
    task_type="SEQ_CLS",
    r=8,
    lora_alpha=16,
    lora_dropout=0.01,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)
model = get_peft_model(model, peft_config)


In [12]:
training_args = TrainingArguments(
    disable_tqdm=False,
    output_dir="./SST2_adaptive_results",
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=2,
    logging_steps=500,
    eval_strategy="steps",
    eval_steps=500,
    bf16=True,              
    report_to="none",
    save_strategy="no",
)


In [13]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [14]:

from torch.utils.data import DataLoader
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]

val_dataloader = DataLoader(
    eval_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=data_collator
)

In [15]:
from adaptive_lora.callbacks import AdaptiveLoRACallback

adaptive_callback = AdaptiveLoRACallback(
    val_dataloader=val_dataloader,
    total_rank=650,
    tau=0.9,      
    verbose=True
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[adaptive_callback],
)

In [17]:


# 12️⃣ Train (should now run cleanly)
trainer.train()


--- AdaptiveLoRA: Preparing ranks for Epoch 1 ---
Computing BI importance scores (pre-training)...


Computing BI scores:   0%|          | 0/109 [00:00<?, ?it/s]

Allocating new ranks based on BI scores...
Applying new ranks to LoRA modules for this epoch...
  - base_model.model.model.layers.0.self_attn.q_proj: r=8 → 7 (Score: 0.9903)
  - base_model.model.model.layers.0.self_attn.k_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.self_attn.v_proj: r=8 → 7 (Score: 0.9978)
  - base_model.model.model.layers.0.self_attn.o_proj: r=8 → 7 (Score: 0.9904)
  - base_model.model.model.layers.1.self_attn.q_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.1.self_attn.k_proj: r=8 → 7 (Score: 0.9926)
  - base_model.model.model.layers.1.self_attn.v_proj: r=8 → 7 (Score: 0.9852)
  - base_model.model.model.layers.1.self_attn.o_proj: r=8 → 7 (Score: 0.9957)
  - base_model.model.model.layers.2.self_attn.q_proj: r=8 → 7 (Score: 0.9922)
  - base_model.model.model.layers.2.self_attn.k_proj: r=8 → 7 (Score: 0.9995)
  - base_model.model.model.layers.2.self_attn.v_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.mode

Step,Training Loss,Validation Loss,Accuracy
500,0.412300,0.462370,{'accuracy': 0.9243119266055045}
1000,0.334400,0.282525,{'accuracy': 0.9311926605504587}
1500,0.295400,0.283762,{'accuracy': 0.9369266055045872}
2000,0.302600,0.283557,{'accuracy': 0.9415137614678899}
2500,0.281600,0.315463,{'accuracy': 0.9403669724770642}
3000,0.270600,0.291247,{'accuracy': 0.9438073394495413}
3500,0.247000,0.285512,{'accuracy': 0.9415137614678899}
4000,0.256300,0.353181,{'accuracy': 0.9392201834862385}
4500,0.266300,0.320993,{'accuracy': 0.944954128440367}
5000,0.255700,0.296338,{'accuracy': 0.9426605504587156}


📄 Epoch 1: Rank allocations logged to ./logs/adaptive_lora_epoch_logs.csv


--- AdaptiveLoRA: Preparing ranks for Epoch 2 ---
Computing BI importance scores (pre-training)...


...

Computing BI scores:   0%|          | 0/109 [00:00<?, ?it/s]

Allocating new ranks based on BI scores...
Applying new ranks to LoRA modules for this epoch...
  - base_model.model.model.layers.0.self_attn.q_proj: r=7 (Unchanged, Score: 0.9903)
  - base_model.model.model.layers.0.self_attn.k_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.self_attn.v_proj: r=7 (Unchanged, Score: 0.9978)
  - base_model.model.model.layers.0.self_attn.o_proj: r=7 (Unchanged, Score: 0.9913)
  - base_model.model.model.layers.1.self_attn.q_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.1.self_attn.k_proj: r=7 (Unchanged, Score: 0.9922)
  - base_model.model.model.layers.1.self_attn.v_proj: r=7 (Unchanged, Score: 0.9851)
  - base_model.model.model.layers.1.self_attn.o_proj: r=7 (Unchanged, Score: 0.9952)
  - base_model.model.model.layers.2.self_attn.q_proj: r=7 (Unchanged, Score: 0.9922)
  - base_model.model.model.layers.2.self_attn.k_proj: r=7 (Unchanged, Score: 0.9995)
  - base_model.model.model.layers.2.self_attn.v_proj: 

Computing BI scores:   0%|          | 0/109 [00:00<?, ?it/s]

Allocating new ranks based on BI scores...
Applying new ranks to LoRA modules for this epoch...
  - base_model.model.model.layers.0.self_attn.q_proj: r=7 (Unchanged, Score: 0.9903)
  - base_model.model.model.layers.0.self_attn.k_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.0.self_attn.v_proj: r=7 (Unchanged, Score: 0.9978)
  - base_model.model.model.layers.0.self_attn.o_proj: r=7 (Unchanged, Score: 0.9914)
  - base_model.model.model.layers.1.self_attn.q_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.model.layers.1.self_attn.k_proj: r=7 (Unchanged, Score: 0.9922)
  - base_model.model.model.layers.1.self_attn.v_proj: r=7 (Unchanged, Score: 0.9851)
  - base_model.model.model.layers.1.self_attn.o_proj: r=7 (Unchanged, Score: 0.9951)
  - base_model.model.model.layers.2.self_attn.q_proj: r=7 (Unchanged, Score: 0.9922)
  - base_model.model.model.layers.2.self_attn.k_proj: r=7 (Unchanged, Score: 0.9994)
  - base_model.model.model.layers.2.self_attn.v_proj: 

TrainOutput(global_step=50514, training_loss=0.2214991523868639, metrics={'train_runtime': 37266.6957, 'train_samples_per_second': 10.843, 'train_steps_per_second': 1.355, 'total_flos': 8.662313818284442e+16, 'train_loss': 0.2214991523868639, 'epoch': 6.0})

In [18]:
met = trainer.evaluate()
print(met)

{'eval_loss': 0.3767743706703186, 'eval_accuracy': {'accuracy': 0.9426605504587156}, 'eval_runtime': 63.4367, 'eval_samples_per_second': 13.746, 'eval_steps_per_second': 6.873, 'epoch': 6.0}
